In [1]:
import pickle
import os
import sensus

import open3d as o3d
import numpy as np

from sensus.utils.data_converter import pc2pc_object
from open3d.web_visualizer import draw
from mmdetection3d import data, demo, configs, checkpoints
from sensus import configs as sensus_configs
from matplotlib import pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2023-04-06 11:09:05,469 - instantiator - Created a temporary directory at /tmp/tmpdib22gz3
INFO - 2023-04-06 11:09:05,470 - instantiator - Writing /tmp/tmpdib22gz3/_remote_module_non_scriptable.py


[Open3D INFO] Resetting default logger to print to terminal.


# Preprocess

In [5]:
# Read pickle file lidar.pickle from data directory
with open(os.path.join(sensus.__path__[0], 'data/lidar.pickle'), 'rb') as f:
    lidar_pickle = pickle.load(f)

In [6]:
pc_buffer = np.frombuffer(lidar_pickle.data,
                    dtype=[('x', np.float32), ('y', np.float32), ('z', np.float32), ('intensity', np.float32)],
                    count=lidar_pickle.width*lidar_pickle.height, offset=0)
pc_ros = pc_buffer.view(dtype=np.float32).reshape(pc_buffer.shape[0], -1)
pc_ros[:, 2] = pc_ros[:, 2] + 4

# Inference

In [7]:
from mmdet3d.apis import inference_detector, init_model
from mmdet3d.utils import register_all_modules

In [8]:
register_all_modules()
# model_cfg = os.path.join(sensus_configs.__path__[0],
#     'second/second_hv_secfpn_8xb6-80e_kitti-3d-3class-ros.py')
model_cfg = os.path.join(configs.__path__[0],
    'centerpoint/centerpoint_voxel0075_second_secfpn_head-circlenms_8xb4-cyclic-20e_nus-3d.py')
# checkpoint_path = os.path.join(checkpoints.__path__[0],
#     'hv_second_secfpn_6x8_80e_kitti-3d-3class_20210831_022017-ae782e87.pth')
checkpoint_path = os.path.join(checkpoints.__path__[0],
    'centerpoint_0075voxel_second_secfpn_circlenms_4x8_cyclic_20e_nus_20220810_011659-04cb3a3b.pth')
device = 'cuda:0'

model = init_model(model_cfg, checkpoint_path, device=device)

Loads checkpoint by local backend from path: /home/messi/alvaro/sensus-loci/mmdetection3d/checkpoints/centerpoint_0075voxel_second_secfpn_circlenms_4x8_cyclic_20e_nus_20220810_011659-04cb3a3b.pth


In [25]:
# Print the max and min values of each column in the point cloud
print(np.max(pc_ros, axis=0))
print(np.min(pc_ros, axis=0))

[115.05504    115.06092     -0.7852335    0.95542234]
[-116.592995   -115.0505       -4.008335      0.61992574]


In [27]:
model.cfg.test_pipeline

[{'type': 'LoadPointsFromFile',
  'coord_type': 'LIDAR',
  'load_dim': 4,
  'use_dim': 5},
 {'type': 'LoadPointsFromMultiSweeps',
  'sweeps_num': 9,
  'use_dim': [0, 1, 2, 3],
  'pad_empty_sweeps': True,
  'remove_close': True},
 {'type': 'MultiScaleFlipAug3D',
  'img_scale': (1333, 800),
  'pts_scale_ratio': 1,
  'flip': False,
  'transforms': [{'type': 'GlobalRotScaleTrans',
    'rot_range': [0, 0],
    'scale_ratio_range': [1.0, 1.0],
    'translation_std': [0, 0, 0]},
   {'type': 'RandomFlip3D'},
   {'type': 'PointsRangeFilter',
    'point_cloud_range': [-54, -54, -5.0, 54, 54, 3.0]}]},
 {'type': 'Pack3DDetInputs', 'keys': ['points']}]

In [9]:
# ROS
pc_object_ros, _ = pc2pc_object(pc_ros.flatten(), model.cfg.test_pipeline)
result_ros, _ = inference_detector(model, pc_object_ros)

04/06 11:13:37 - mmengine - WARNING - The "transform" registry in mmdet3d did not set import location. Fallback to call `mmdet3d.utils.register_all_modules` instead.


/home/messi/alvaro/sensus-loci/mmdetection3d/mmdet3d/models/task_modules/coders/centerpoint_bbox_coders.py:201: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.post_center_range = torch.tensor(
/home/messi/alvaro/sensus-loci/mmdetection3d/mmdet3d/models/task_modules/coders/centerpoint_bbox_coders.py:201: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.post_center_range = torch.tensor(


In [10]:
type(result_ros)

mmdet3d.structures.det3d_data_sample.Det3DDataSample

In [24]:
for i, bbox_pred in enumerate(result_ros.pred_instances_3d.bboxes_3d.tensor.tolist()):
    print(bbox_pred)
    print(result_ros.pred_instances_3d.scores_3d[i].item())
    break

[22.506675720214844, 28.6533203125, -3.9237818717956543, 4.577454566955566, 1.8992881774902344, 1.6064486503601074, 2.5990734100341797, -2.3876028060913086, 2.019270181655884]
0.8311319351196289


In [16]:
result_ros.pred_instances_3d.labels_3d.tolist()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 5]

In [55]:
result_ros.pred_instances_3d.bboxes_3d.tensor.min(axis=0)

torch.return_types.min(
values=tensor([-32.6744, -36.7824,  -4.2120,   0.4797,   0.7362,   0.8834,  -3.0265,
         -7.6658,  -9.6750], device='cuda:0'),
indices=tensor([ 4, 17, 10, 22, 23, 21, 19, 11,  7], device='cuda:0'))

In [ ]:
result_ros.pred_instances_3d.bboxes_3d.tensor.max(axis=0)

torch.return_types.max(
values=tensor([37.5542, 31.8943,  0.0938, 17.5359,  3.1787,  4.2872,  3.1106,  6.4032,
         6.4455], device='cuda:0'),
indices=tensor([20, 10, 23, 20, 21, 20, 21, 15,  3], device='cuda:0'))

In [30]:
# print(result_ros.pred_instances_3d.labels_3d)
# print(result_ros.pred_instances_3d.scores_3d)
# print(result_ros.pred_instances_3d.bboxes_3d)
print(result_ros.pred_instances_3d.bboxes_3d.tensor[0])
print(result_ros.pred_instances_3d.bboxes_3d.tensor.shape)

tensor([  1.6682, -37.9861,  -2.6241,   4.3703,   1.7161,   1.5333,   3.2115],
       device='cuda:0')
torch.Size([18, 7])


In [8]:
# Same result after processing points (maybe processing under the hood when
# using np.array pc)
points = points.reshape(-1, 4)
result, data = inference_detector(model, points)

AttributeError: 'numpy.ndarray' object has no attribute 'rotate'

In [71]:
print(result.pred_instances_3d.labels_3d)
print(result.pred_instances_3d.scores_3d)
print(result.pred_instances_3d.bboxes_3d.tensor.shape)

tensor([0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([0.1304, 0.1173, 0.3624, 0.3572, 0.9088, 0.8917, 0.8498, 0.7471, 0.7245,
        0.6414, 0.6307, 0.5603, 0.4316, 0.3937], device='cuda:0')
torch.Size([14, 7])


# Visualization

In [11]:
bin_path = os.path.join(demo.__path__[0],
    'data/kitti/000008.bin')
pc_path = os.path.join(
    '/home/messi/alvaro/sensus-loci/mmdetection3d/demo/kitti_pointpillars/kitti_000008',
    'kitti_000008_points.obj')
bboxes_path = os.path.join(
    '/home/messi/alvaro/sensus-loci/mmdetection3d/demo/kitti_pointpillars/kitti_000008',
    'kitti_000008_pred.obj')

In [12]:
# Read pc from bin file
with open(bin_path, 'rb') as f:
    points = np.fromfile(f, dtype=np.float32, count=-1).reshape([-1, 4])

pcd_bin = o3d.geometry.PointCloud()
pcd_bin.points = o3d.utility.Vector3dVector(points[:, :3])
print(pcd_bin)

PointCloud with 17238 points.


In [5]:
# Read pc from obj file (reading with o3d.io.read_triangle_mesh or 
# read_point_cloud does not work)
pc = []
with open(pc_path, 'rb') as f:
    for each in f.readlines():
        p1, p2, p3 = each.decode('utf-8').split(' ')[1:]
        pc.append([float(p1), float(p2), float(p3.replace('\n', ''))])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.array(pc))
print(pcd)

bboxes = o3d.io.read_triangle_mesh(bboxes_path)
bboxes.compute_vertex_normals()     # For solid rendering with lighting
bboxes_lines = o3d.geometry.LineSet().create_from_triangle_mesh(bboxes)
bboxes_lines.paint_uniform_color([1, 0, 0])
print(bboxes_lines)

PointCloud with 16897 points.
LineSet with 360 lines.


In [29]:
draw([pcd, bboxes_lines], width=900, height=600, point_size=2)

WebVisualizer(window_uid='window_11')

In [13]:
draw([pcd_bin, bboxes_lines], width=900, height=600, point_size=2)

WebVisualizer(window_uid='window_0')